![alt text](https://storage.googleapis.com/pr-newsroom-wp/1/2018/11/Spotify_Logo_CMYK_Green.png)

## Working with Spotify Data using SpotiPy
-Jeremy Seibert

Spotify is an amazing music streaming service that touches over 200 million customers. This is the notebook where I will show you how to work with the Spotify API using SpotiPy. This was the methodology that I used to create the datasets used in my spotify song prediction notebook. The SpotiPy package assists in the API connection to the Spotify backend's publicly available data. Once connected to the API, we can query an enormous amount of Spotify data related to their user, playlist, and song data. So lets jump in!

### 1. Import the Necessities

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import pandas as pd

### 2. Setup your Spotify App Credentials

As with most APIs, you will need to register with the service that you are wanting to connect too. This is exactly the case for Spotify. Spotify's Developers have made it easy for other developers to connect to the to their data. 

1. You will first need to use your Spotify account to register for a developer account, or you can setup a free spotify account and then register for your developer account [Setup Spotify Dev Account here](https://developer.spotify.com/dashboard/). 

2. After registering setring up your developer account, you will need to Setup an App. 
    - Locate the Dashboard tab, and click it. 
    - Select "Create a Client ID"
    - Fill in the name of the App, in this case I just put "Notebook 1"
    - Fill in the App or Hardware Description, I put "Jupyter Notebook"
    - Select what you are building, I put "Website"
    - The question regarding a commercial integration simply means are yo trying to make money off of this? & the quick answer here is No (If you are a business, and are looking to integrate this to make money, check Yes)
    - Agree to the Terms of Service.
    
    
3. You should be redirected to your Dashboard, once there click the "Edit settings" button. 

4. In the Edit Settings popup, find the Redirect URIs blank and insert "http://google.com/" and MAKE SURE AT THE BOTTOM YOU CLICK SAVE. 

5. Go back to your dashboard an locate your Client ID and your Secret ID and fill them into the notebook. 

In [2]:
cid ='<Your App ID>'
secret = '<Your Secret App ID>'
username = "<Your User ID>"

In [ ]:
cid ='77e685881f74452c83f138c8868f6202'
secret = 'b39d5ef2444543fe867b2db80b69b962'
username = '1249424214'

In [ ]:
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
scope = 'user-library-read playlist-read-private'
token = util.prompt_for_user_token(username, scope,client_id=cid,client_secret=secret,redirect_uri='http://spotify.com/')
if token is not None:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

In [ ]:
def get_tracks(user,playlist):
    ''' This function takes a user key and playlistID and returns the trackIDs within that users playlist.'''
    uplaylist = sp.user_playlist(user,playlist)
    tracks = uplaylist["tracks"]
    songs = tracks["items"] 
    while tracks['next']:
        tracks = sp.next(tracks)
        for item in tracks["items"]:
            songs.append(item)
    ids = [songs[i]['track']['id'] for i in range(len(songs))]
    return ids

In [ ]:
def get_playlists():
    '''This function specifically calls the playlist of Good (Liked) Songs, and the Bad (Unliked) Songs.'''
    good_ids = get_tracks("<Your User ID>","<Your Good Playlist ID>")
    bad_ids = get_tracks("<Your User ID>","<Your Bad Playlist ID>")
    return good_ids,bad_ids

In [ ]:
def get_audio_features(ids,target_val):
    '''This function takes a track and gathers the track's Audio Features, and adds a new target feature column.'''
    features = []
    for i in range(0,len(ids),50):
        audio_features = sp.audio_features(ids[i:i+50])
        for track in audio_features:
            features.append(track)
            features[-1]['target'] = target_val
    return features

In [ ]:
def get_playlist_features():
    '''This function calls the get_audio_features() function and declares the target features.'''
    good_ids, bad_ids = get_playlists()
    good_features = get_audio_features(good_ids,1)
    bad_features = get_audio_features(bad_ids,0)
    return good_features, bad_features

In [ ]:
good_features, bad_features = get_playlist_features()
good_data = pd.DataFrame(good_features)
bad_data = pd.DataFrame(bad_features)
total_data = pd.concat([good_data,bad_data], ignore_index=True)
total_data.head()